In [2]:
# KITTI-360 PLY to High-Quality VLM Images
# Creates premium quality images optimized for Vision Language Model analysis

import numpy as np
import matplotlib.pyplot as plt
import cv2
import struct
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Try to import Open3D
try:
    import open3d as o3d
    HAS_OPEN3D = True
    print("✓ Using Open3D for robust PLY loading")
except ImportError:
    HAS_OPEN3D = False
    print("⚠ Using basic PLY parser")

def load_kitti360_ply(ply_path: str):
    """Load KITTI-360 PLY file with robust error handling"""
    
    if HAS_OPEN3D:
        try:
            pcd = o3d.io.read_point_cloud(ply_path)
            points = np.asarray(pcd.points)
            
            colors = None
            if pcd.has_colors():
                colors = np.asarray(pcd.colors) * 255
            
            print(f"✓ Loaded {len(points):,} points using Open3D")
            return points, colors
                
        except Exception as e:
            print(f"Open3D failed: {e}")
            print("Falling back to basic parser...")
    
    # Basic PLY parser
    try:
        with open(ply_path, 'rb') as f:
            # Read header
            header_lines = []
            while True:
                line = f.readline().decode('ascii', errors='ignore').strip()
                header_lines.append(line)
                if line == 'end_header':
                    break
            
            # Parse header info
            vertex_count = 0
            properties = []
            is_binary = False
            
            for line in header_lines:
                if line.startswith('element vertex'):
                    vertex_count = int(line.split()[-1])
                elif line.startswith('property'):
                    properties.append(line.split()[1:])
                elif 'binary' in line:
                    is_binary = True
            
            print(f"PLY format: {'Binary' if is_binary else 'ASCII'}")
            print(f"Vertices: {vertex_count:,}")
            
            # Read data based on format
            if is_binary:
                points, colors = read_binary_ply_data(f, vertex_count, properties)
            else:
                points, colors = read_ascii_ply_data(f, vertex_count)
            
            if points is not None:
                print(f"✓ Loaded {len(points):,} points using basic parser")
            
            return points, colors
            
    except Exception as e:
        print(f"Error loading PLY file: {e}")
        return None, None

def read_binary_ply_data(f, vertex_count, properties):
    """Read binary PLY data efficiently"""
    data = []
    
    for i in range(vertex_count):
        row = []
        for prop in properties:
            if len(prop) < 2:
                continue
                
            prop_type = prop[0]
            
            if prop_type in ['float', 'float32']:
                val = struct.unpack('<f', f.read(4))[0]
            elif prop_type in ['double', 'float64']:
                val = struct.unpack('<d', f.read(8))[0]
            elif prop_type in ['uchar', 'uint8']:
                val = struct.unpack('<B', f.read(1))[0]
            elif prop_type in ['int', 'int32']:
                val = struct.unpack('<i', f.read(4))[0]
            else:
                val = 0
                
            row.append(val)
        
        data.append(row)
        
        if i % 50000 == 0:
            print(f"Reading: {i:,}/{vertex_count:,}")
    
    data = np.array(data)
    points = data[:, :3]
    colors = data[:, 3:6] if data.shape[1] >= 6 else None
    
    return points, colors

def read_ascii_ply_data(f, vertex_count):
    """Read ASCII PLY data efficiently"""
    data = []
    
    for i in range(vertex_count):
        line = f.readline().decode('ascii', errors='ignore').strip()
        if line:
            values = list(map(float, line.split()))
            data.append(values)
        
        if i % 50000 == 0:
            print(f"Reading: {i:,}/{vertex_count:,}")
    
    data = np.array(data)
    points = data[:, :3]
    colors = data[:, 3:6] if data.shape[1] >= 6 else None
    
    return points, colors

def create_premium_bev_image(points: np.ndarray, colors: np.ndarray = None, 
                           image_size: int = 2048, xy_range: float = 60) -> np.ndarray:
    """Create premium quality Bird's Eye View for VLM analysis"""
    
    print("Creating premium BEV image...")
    
    # Get actual coordinate ranges for this dataset
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    z_min, z_max = np.min(points[:, 2]), np.max(points[:, 2])
    
    print(f"  Coordinate ranges: X[{x_min:.1f}, {x_max:.1f}], Y[{y_min:.1f}, {y_max:.1f}], Z[{z_min:.1f}, {z_max:.1f}]")
    
    # Adaptive height filtering based on actual data range
    z_span = z_max - z_min
    if z_span > 10:  # Large buildings/structures
        ground_threshold = z_min + z_span * 0.1
        low_threshold = z_min + z_span * 0.3
        mid_threshold = z_min + z_span * 0.6
        high_threshold = z_min + z_span * 0.9
    else:  # More typical scene
        ground_threshold = z_min + 2
        low_threshold = z_min + 4
        mid_threshold = z_min + 8
        high_threshold = z_min + 12
    
    ground_mask = (points[:, 2] >= z_min) & (points[:, 2] <= ground_threshold)
    low_mask = (points[:, 2] > ground_threshold) & (points[:, 2] <= low_threshold)
    mid_mask = (points[:, 2] > low_threshold) & (points[:, 2] <= mid_threshold)
    high_mask = (points[:, 2] > mid_threshold) & (points[:, 2] <= z_max)
    
    print(f"  Height layers: Ground({np.sum(ground_mask)}), Low({np.sum(low_mask)}), Mid({np.sum(mid_mask)}), High({np.sum(high_mask)})")
    
    # Create high-resolution BEV
    bev_image = np.zeros((image_size, image_size, 3), dtype=np.uint8)
    
    def add_layer_to_bev(point_subset, color_rgb, alpha=0.8):
        if len(point_subset) == 0:
            return
            
        # Convert to image coordinates using actual coordinate ranges
        x_img = ((point_subset[:, 0] - x_min) / (x_max - x_min) * image_size).astype(int)
        y_img = ((point_subset[:, 1] - y_min) / (y_max - y_min) * image_size).astype(int)
        
        # Keep valid points
        valid_mask = (x_img >= 0) & (x_img < image_size) & (y_img >= 0) & (y_img < image_size)
        x_valid = x_img[valid_mask]
        y_valid = y_img[valid_mask]
        
        print(f"    Valid points for this layer: {len(x_valid)}")
        
        # Add points with specified color (more efficient vectorized approach)
        if len(x_valid) > 0:
            for x, y in zip(x_valid, y_valid):
                # Blend with existing pixels for better visualization
                existing = bev_image[y, x].astype(float)
                new_color = np.array(color_rgb, dtype=float)
                blended = (1 - alpha) * existing + alpha * new_color
                bev_image[y, x] = np.clip(blended, 0, 255).astype(np.uint8)
    
    # Add layers with semantic colors
    add_layer_to_bev(points[ground_mask], [60, 60, 60], alpha=0.9)      # Dark gray for ground
    add_layer_to_bev(points[low_mask], [255, 200, 0], alpha=0.8)        # Yellow for vehicles/objects
    add_layer_to_bev(points[mid_mask], [0, 150, 255], alpha=0.7)        # Blue for mid-level structures
    add_layer_to_bev(points[high_mask], [255, 100, 100], alpha=0.6)     # Red for high structures
    
    # Apply Gaussian blur for smoother appearance
    bev_image = cv2.GaussianBlur(bev_image, (3, 3), 0)
    
    # Enhance contrast
    bev_image = cv2.convertScaleAbs(bev_image, alpha=1.2, beta=10)
    
    return bev_image

def create_premium_range_image(points: np.ndarray, image_size: int = 2048) -> np.ndarray:
    """Create premium range image with enhanced features"""
    
    print("Creating premium range image...")
    
    # Center the coordinates for better spherical projection
    x_center = (np.min(points[:, 0]) + np.max(points[:, 0])) / 2
    y_center = (np.min(points[:, 1]) + np.max(points[:, 1])) / 2
    z_center = (np.min(points[:, 2]) + np.max(points[:, 2])) / 2
    
    # Translate to center
    x = points[:, 0] - x_center
    y = points[:, 1] - y_center  
    z = points[:, 2] - z_center
    
    print(f"  Centered coordinates: X[{np.min(x):.1f}, {np.max(x):.1f}], Y[{np.min(y):.1f}, {np.max(y):.1f}], Z[{np.min(z):.1f}, {np.max(z):.1f}]")
    
    # Calculate spherical coordinates
    range_vals = np.sqrt(x**2 + y**2 + z**2)
    azimuth = np.arctan2(y, x)  # -π to π
    elevation = np.arctan2(z, np.sqrt(x**2 + y**2))  # -π/2 to π/2
    
    print(f"  Range values: [{np.min(range_vals):.1f}, {np.max(range_vals):.1f}]")
    print(f"  Azimuth range: [{np.min(azimuth):.2f}, {np.max(azimuth):.2f}] radians")
    print(f"  Elevation range: [{np.min(elevation):.2f}, {np.max(elevation):.2f}] radians")
    
    # Enhanced image coordinate mapping
    h_img = ((azimuth + np.pi) / (2 * np.pi) * image_size).astype(int)
    v_img = ((elevation + np.pi/2) / np.pi * image_size).astype(int)
    
    # Filter valid pixels
    max_range = np.percentile(range_vals, 95)  # Use 95th percentile as max range
    valid_mask = (h_img >= 0) & (h_img < image_size) & (v_img >= 0) & (v_img < image_size) & (range_vals <= max_range)
    h_valid = h_img[valid_mask]
    v_valid = v_img[valid_mask]
    range_valid = range_vals[valid_mask]
    
    print(f"  Valid pixels after filtering: {len(h_valid)}")
    
    # Create high-resolution range image
    range_image = np.zeros((image_size, image_size), dtype=np.float32)
    count_image = np.zeros((image_size, image_size), dtype=np.float32)
    
    # Accumulate ranges for averaging (handles multiple points per pixel)
    for h, v, r in zip(h_valid, v_valid, range_valid):
        range_image[v, h] += r
        count_image[v, h] += 1
    
    # Average overlapping points
    mask = count_image > 0
    if np.sum(mask) > 0:
        range_image[mask] /= count_image[mask]
        
        # Apply median filter to reduce noise
        range_image = cv2.medianBlur(range_image.astype(np.float32), 3)
        
        # Enhanced colorization
        range_max = np.max(range_image[range_image > 0])
        if range_max > 0:
            range_normalized = np.clip(range_image / range_max * 255, 0, 255).astype(np.uint8)
        else:
            range_normalized = np.zeros_like(range_image, dtype=np.uint8)
    else:
        print("  Warning: No valid range data found, creating empty image")
        range_normalized = np.zeros((image_size, image_size), dtype=np.uint8)
    
    # Use a better colormap
    range_rgb = cv2.applyColorMap(range_normalized, cv2.COLORMAP_TURBO)
    
    # Enhance contrast and brightness
    range_rgb = cv2.convertScaleAbs(range_rgb, alpha=1.1, beta=5)
    
    return range_rgb

def create_semantic_density_image(points: np.ndarray, image_size: int = 2048, xy_range: float = 60) -> np.ndarray:
    """Create semantic density image showing object distributions"""
    
    print("Creating semantic density image...")
    
    # Get actual coordinate ranges
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    z_min, z_max = np.min(points[:, 2]), np.max(points[:, 2])
    
    # Adaptive height layers based on actual data
    z_span = z_max - z_min
    if z_span > 10:  # Large buildings/structures
        ground_threshold = z_min + z_span * 0.2
        object_threshold = z_min + z_span * 0.5
    else:  # More typical scene
        ground_threshold = z_min + 2
        object_threshold = z_min + 6
    
    # Create multi-layer density maps
    layers = {
        'ground': (points[:, 2] >= z_min) & (points[:, 2] <= ground_threshold),
        'objects': (points[:, 2] > ground_threshold) & (points[:, 2] <= object_threshold),
        'structures': (points[:, 2] > object_threshold)
    }
    
    print(f"  Layer distribution: Ground({np.sum(layers['ground'])}), Objects({np.sum(layers['objects'])}), Structures({np.sum(layers['structures'])})")
    
    final_image = np.zeros((image_size, image_size, 3), dtype=np.float32)
    
    for layer_name, mask in layers.items():
        if not mask.any():
            continue
            
        layer_points = points[mask]
        
        # Create density grid
        grid_size = 400  # High resolution internal grid
        density_grid = np.zeros((grid_size, grid_size), dtype=np.float32)
        
        # Map points to grid using actual coordinate ranges
        x_grid = ((layer_points[:, 0] - x_min) / (x_max - x_min) * grid_size).astype(int)
        y_grid = ((layer_points[:, 1] - y_min) / (y_max - y_min) * grid_size).astype(int)
        
        # Count points in each cell
        valid_mask = (x_grid >= 0) & (x_grid < grid_size) & (y_grid >= 0) & (y_grid < grid_size)
        x_valid = x_grid[valid_mask]
        y_valid = y_grid[valid_mask]
        
        for x, y in zip(x_valid, y_valid):
            density_grid[y, x] += 1
        
        # Apply Gaussian smoothing
        density_grid = cv2.GaussianBlur(density_grid, (15, 15), 0)
        
        # Resize to final image size
        density_resized = cv2.resize(density_grid, (image_size, image_size))
        
        # Normalize
        if np.max(density_resized) > 0:
            density_normalized = density_resized / np.max(density_resized)
        else:
            density_normalized = density_resized
        
        # Assign to color channels
        if layer_name == 'ground':
            final_image[:, :, 2] += density_normalized * 255  # Blue channel
        elif layer_name == 'objects':
            final_image[:, :, 1] += density_normalized * 255  # Green channel
        else:  # structures
            final_image[:, :, 0] += density_normalized * 255  # Red channel
    
    # Normalize and convert to uint8
    final_image = np.clip(final_image, 0, 255).astype(np.uint8)
    
    # Enhance contrast
    final_image = cv2.convertScaleAbs(final_image, alpha=1.1, beta=5)
    
    return final_image

def create_composite_vlm_image(points: np.ndarray, colors: np.ndarray = None, 
                              image_size: int = 1024) -> np.ndarray:
    """Create a composite image combining multiple views for comprehensive VLM analysis"""
    
    print("Creating composite VLM image...")
    
    # Get coordinate ranges for adaptive processing
    x_range = np.max(points[:, 0]) - np.min(points[:, 0])
    y_range = np.max(points[:, 1]) - np.min(points[:, 1])
    max_range = max(x_range, y_range)
    
    print(f"  Scene dimensions: {x_range:.1f}m x {y_range:.1f}m")
    
    # Create individual high-quality images
    bev = create_premium_bev_image(points, colors, image_size, xy_range=max_range/2)
    range_img = create_premium_range_image(points, image_size)
    density = create_semantic_density_image(points, image_size, xy_range=max_range/2)
    
    # Resize for composite (use half size for each quadrant)
    half_size = image_size // 2
    bev_small = cv2.resize(bev, (half_size, half_size))
    range_small = cv2.resize(range_img, (half_size, half_size))
    density_small = cv2.resize(density, (half_size, half_size))
    
    # Create height histogram as an image
    hist, bins = np.histogram(points[:, 2], bins=50, range=(-5, 20))
    hist_img = np.zeros((half_size, half_size, 3), dtype=np.uint8)
    
    # Draw histogram
    hist_normalized = hist / np.max(hist) * (half_size - 20)
    bin_width = half_size // len(hist)
    
    for i, h in enumerate(hist_normalized):
        x = i * bin_width
        y_start = half_size - int(h) - 10
        y_end = half_size - 10
        cv2.rectangle(hist_img, (x, y_start), (x + bin_width - 1, y_end), (0, 255, 255), -1)
    
    # Add labels and grid
    cv2.putText(hist_img, "Height Distribution", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Combine into composite
    composite = np.zeros((image_size, image_size, 3), dtype=np.uint8)
    
    # Top row: BEV and Range
    composite[:half_size, :half_size] = bev_small
    composite[:half_size, half_size:] = range_small
    
    # Bottom row: Density and Stats
    composite[half_size:, :half_size] = density_small
    composite[half_size:, half_size:] = hist_img
    
    # Add border lines
    cv2.line(composite, (half_size, 0), (half_size, image_size), (255, 255, 255), 2)
    cv2.line(composite, (0, half_size), (image_size, half_size), (255, 255, 255), 2)
    
    # Add labels
    cv2.putText(composite, "Bird's Eye View", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(composite, "Range Image", (half_size + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(composite, "Semantic Density", (10, half_size + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    return composite, bev, range_img, density

def process_ply_for_vlm(ply_path: str):
    """Process PLY file and create premium images for VLM analysis"""
    
    print("KITTI-360 Premium VLM Image Generator")
    print("=" * 50)
    print(f"Processing: {Path(ply_path).name}")
    
    # Load point cloud
    points, colors = load_kitti360_ply(ply_path)
    
    if points is None:
        print("❌ Failed to load point cloud")
        return None
    
    # Print detailed statistics
    print(f"\n📊 Point Cloud Analysis:")
    print(f"   Total points: {len(points):,}")
    print(f"   X range: {np.min(points[:, 0]):.1f} to {np.max(points[:, 0]):.1f} m ({np.max(points[:, 0])-np.min(points[:, 0]):.1f} m width)")
    print(f"   Y range: {np.min(points[:, 1]):.1f} to {np.max(points[:, 1]):.1f} m ({np.max(points[:, 1])-np.min(points[:, 1]):.1f} m depth)")
    print(f"   Z range: {np.min(points[:, 2]):.1f} to {np.max(points[:, 2]):.1f} m ({np.max(points[:, 2])-np.min(points[:, 2]):.1f} m height)")
    
    # Analyze content with adaptive thresholds
    z_min, z_max = np.min(points[:, 2]), np.max(points[:, 2])
    z_span = z_max - z_min
    
    if z_span > 10:  # Large buildings/structures
        ground_threshold = z_min + z_span * 0.1
        object_threshold = z_min + z_span * 0.3
    else:  # More typical scene
        ground_threshold = z_min + 2
        object_threshold = z_min + 6
    
    ground_points = np.sum((points[:, 2] >= z_min) & (points[:, 2] <= ground_threshold))
    object_points = np.sum((points[:, 2] > ground_threshold) & (points[:, 2] <= object_threshold))
    structure_points = np.sum(points[:, 2] > object_threshold)
    
    print(f"\n🏗️  Scene Composition (Adaptive Thresholds):")
    print(f"   Ground level ({z_min:.1f} to {ground_threshold:.1f}m): {ground_points:,} points ({ground_points/len(points)*100:.1f}%)")
    print(f"   Object level ({ground_threshold:.1f} to {object_threshold:.1f}m):  {object_points:,} points ({object_points/len(points)*100:.1f}%)")
    print(f"   Structure level (>{object_threshold:.1f}m):     {structure_points:,} points ({structure_points/len(points)*100:.1f}%)")
    
    print(f"\n🎨 Generating premium VLM images...")
    
    # Create premium images
    composite, bev, range_img, density = create_composite_vlm_image(points, colors, image_size=1024)
    
    # Also create high-res individual images
    print("Creating high-resolution individual images...")
    x_range = np.max(points[:, 0]) - np.min(points[:, 0])
    y_range = np.max(points[:, 1]) - np.min(points[:, 1])
    max_range = max(x_range, y_range)
    bev_hires = create_premium_bev_image(points, colors, image_size=2048, xy_range=max_range/2)
    
    # Save images
    output_dir = Path("vlm_images")
    output_dir.mkdir(exist_ok=True)
    
    filename_base = Path(ply_path).stem
    
    # Save all versions
    cv2.imwrite(str(output_dir / f"{filename_base}_composite_vlm.jpg"), composite)
    cv2.imwrite(str(output_dir / f"{filename_base}_bev_premium.jpg"), bev_hires)
    cv2.imwrite(str(output_dir / f"{filename_base}_range_premium.jpg"), range_img)
    cv2.imwrite(str(output_dir / f"{filename_base}_density_semantic.jpg"), density)
    
    print(f"\n✅ Premium VLM images saved to {output_dir}/")
    print(f"   📸 {filename_base}_composite_vlm.jpg (1024x1024) - BEST FOR VLM")
    print(f"   📸 {filename_base}_bev_premium.jpg (2048x2048) - High-res BEV")
    print(f"   📸 {filename_base}_range_premium.jpg (2048x2048) - High-res Range")
    print(f"   📸 {filename_base}_density_semantic.jpg (2048x2048) - Semantic Density")
    
    # Display the composite image
    plt.figure(figsize=(15, 15))
    plt.imshow(cv2.cvtColor(composite, cv2.COLOR_BGR2RGB))
    plt.title(f'Premium VLM Image: {filename_base}', fontsize=16, fontweight='bold')
    plt.axis('off')
    
    # Add comprehensive description
    description_text = f"""
KITTI-360 Scene Analysis: {filename_base}

COMPREHENSIVE VLM IMAGE (1024x1024):
✅ Top-Left: Bird's Eye View (Adaptive Height Layers)
✅ Top-Right: Range/Depth Image (Spherical Projection)  
✅ Bottom-Left: Semantic Density (Multi-layer Object Distribution)
✅ Bottom-Right: Height Distribution Analysis

Scene Statistics:
• Total Points: {len(points):,}
• Scene Dimensions: {np.max(points[:, 0])-np.min(points[:, 0]):.1f}m × {np.max(points[:, 1])-np.min(points[:, 1]):.1f}m × {np.max(points[:, 2])-np.min(points[:, 2]):.1f}m
• Ground Coverage: {ground_points/len(points)*100:.1f}% (Z: {z_min:.1f}-{ground_threshold:.1f}m)
• Object Density: {object_points/len(points)*100:.1f}% (Z: {ground_threshold:.1f}-{object_threshold:.1f}m)
• Structure Coverage: {structure_points/len(points)*100:.1f}% (Z: >{object_threshold:.1f}m)

OPTIMIZED FOR VLM ANALYSIS:
🎯 Adaptive coordinate transformation for any scale
🎯 Multi-perspective comprehensive view
🎯 Clear structural and spatial relationships
🎯 Enhanced feature visibility
"""
    
    plt.figtext(0.02, 0.02, description_text, fontsize=10, fontfamily='monospace',
                bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🎯 RECOMMENDED FOR VLM: Use '{filename_base}_composite_vlm.jpg'")
    print("   This image contains comprehensive scene information in a single optimized view.")
    
    return composite, points

# Run the premium conversion
if __name__ == "__main__":
    ply_file_path = r"C:\Users\caleb\OneDrive\Desktop\KITTI_3D\data_3d_semantics\data_3d_semantics\train\2013_05_28_drive_0000_sync\static\0000000002_0000000385.ply"
    
    composite_image, point_cloud = process_ply_for_vlm(ply_file_path)
    
    print("\n" + "=" * 50)
    print("🚀 READY FOR VLM PROCESSING!")
    print("=" * 50)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
✓ Using Open3D for robust PLY loading
KITTI-360 Premium VLM Image Generator
Processing: 0000000002_0000000385.ply
✓ Loaded 3,201,318 points using Open3D

📊 Point Cloud Analysis:
   Total points: 3,201,318
   X range: 827.1 to 1007.7 m (180.6 m width)
   Y range: 3670.5 to 3831.3 m (160.8 m depth)
   Z range: 112.4 to 129.2 m (16.7 m height)

🏗️  Scene Composition (Adaptive Thresholds):
   Ground level (112.4 to 114.1m): 10,506 points (0.3%)
   Object level (114.1 to 117.4m):  2,797,946 points (87.4%)
   Structure level (>117.4m):     392,866 points (12.3%)

🎨 Generating premium VLM images...
Creating composite VLM image...
  Scene dimensions: 180.6m x 160.8m
Creating premium BEV image...
  Coordinate ranges: X[827.1, 1007.7], Y[3670.5, 3831.3], Z[112.4, 129.2]
  Height layers: Ground(10506), Low(2797946), Mid(374227), H

ValueError: cannot convert float NaN to integer